# PAPERMILL

In [1]:
# model's seeds
s1 = 0
s2 = None
s3 = None
s4 = None
s5 = None
s6 = None
s7 = None
s8 = None
s9 = None
s10 = None

# additional paramters
model_parameter_name = 'sl'
GPU_ID = '0'

In [2]:
seeds_list = [int(s) for s in [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10] if s is not None]

assert len(seeds_list) > 0, "seeds_list should have at least one entry"
assert all(isinstance(seed, int) for seed in seeds_list), "Not all entries are integers"
assert model_parameter_name is not None, "model_parameter_name should not be None"
assert GPU_ID is not None, "GPU_ID should not be None"

print("Papermill seeds parameters are: " + str(seeds_list))
print("Papermill model name is: " + model_parameter_name)
print("Papermill GPU_ID is: " + GPU_ID)

Papermill seeds parameters are: [0]
Papermill model name is: sl
Papermill GPU_ID is: 0


# IMPORTS

In [3]:
import sys
import os

os.environ['CUDA_VISIBLE_DEVICES'] = GPU_ID
sys.path.append(os.path.abspath(".."))       # for 'protonet_mnist_add_utils' folder
sys.path.append(os.path.abspath("../.."))    # for 'data' folder
sys.path.append(os.path.abspath("../../..")) # for 'models' and 'datasets' folders


print(sys.path)

['/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation/kandinsky/notebooks', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python38.zip', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8/lib-dynload', '', '/users-1/eleonora/.local/lib/python3.8/site-packages', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8/site-packages', '/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation/kandinsky', '/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation', '/users-1/eleonora/reasoning-shortcuts/IXShort']


In [4]:
import cv2
import torch
import argparse
import datetime
import importlib
import setproctitle, socket, uuid
import torch.nn.functional as F

from tqdm import tqdm
from argparse import Namespace
from numpy import float32, zeros
from datasets import get_dataset
from models import get_model
from models.mnistdpl import MnistDPL
from warmup_scheduler import GradualWarmupScheduler
from datasets.utils.base_dataset import BaseDataset
from torchvision import datasets, transforms
from utils import fprint
from utils.train import train
from utils.test import test
from utils.preprocess_resnet import preprocess
from utils.conf import *
from utils.args import *
from utils.status import progress_bar
from utils.checkpoint import save_model, create_load_ckpt
from utils.dpl_loss import ADDMNIST_DPL
from utils.metrics import (
    evaluate_metrics,
    evaluate_mix,
    mean_entropy,
)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_samples
from scipy.spatial.distance import cdist

from protonet_kand_modules.arguments import args_dpl, args_sl, args_ltn
from protonet_kand_modules.utility_modules.check_gpu import my_gpu_info

# SETUP

In [5]:
if model_parameter_name == 'dpl':   args = args_dpl
elif model_parameter_name == 'sl':  args = args_sl
else:                               args = args_ltn

# saving
save_folder = "kand" 
save_model_name = model_parameter_name
save_paths = []
save_path = os.path.join("..", "outputs", 
    save_folder, 
    "baseline-DEBUG", 
    save_model_name
)
save_paths.append(save_path)
print(f"Save paths: {str(save_paths)}")

if args.model in ['prokandsl', 'prokandltn', 'prokanddpl'] or args.prototypes:
    raise ValueError("This experiment is NOT meant for pNet based models.")

Save paths: ['../outputs/kand/baseline-DEBUG/sl']


In [6]:
my_gpu_info()

Torch version:  1.13.0+cu117
CUDA version:  11.7
Number of GPUs available: 1
Device 0: NVIDIA GeForce GTX TITAN X
  Memory Allocated: 0 bytes
  Memory Cached: 0 bytes


In [7]:
# Add uuid, timestamp and hostname for logging
args.conf_jobnum = str(uuid.uuid4())
args.conf_timestamp = str(datetime.datetime.now())
args.conf_host = socket.gethostname()

# set job name
setproctitle.setproctitle(
    "{}_{}_{}".format(
        args.model,
        args.buffer_size if "buffer_size" in args else 0,
        args.dataset,
    )
)

# DATASET

In [8]:
dataset = get_dataset(args)
n_images, c_split = dataset.get_split()
encoder, decoder = dataset.get_backbone()
model = get_model(args, encoder, decoder, n_images, c_split)
loss = model.get_loss(args)
model.start_optim(args)

print("Using Dataset: ", dataset)
print("Number of images: ", n_images)
print("Using backbone: ", encoder)
print("Using Model: ", model)
print("Using Loss: ", loss)
print("Working with taks: ", args.task)

Available datasets: ['mnmath', 'xor', 'clipboia', 'shortmnist', 'restrictedmnist', 'minikandinsky', 'presddoia', 'prekandinsky', 'sddoia', 'clipkandinsky', 'addmnist', 'clipshortmnist', 'boia_original', 'boia_original_embedded', 'clipsddoia', 'boia', 'kandinsky', 'halfmnist']
kand says Namespace(and_op='Prod', backbone='conceptizer', batch_size=32, beta=0.99, boia_model='ce', boia_ood_knowledge=False, c_sup=0.0, c_sup_ltn=0, checkin=None, checkout=False, classes_per_it=3, concept_extractor_path='ultralytics/finetuned/kand_best_100.pt', conf_host='pssr', conf_jobnum='2bcb2bf6-5162-4a95-8f05-2b6bd2749bb0', conf_timestamp='2025-08-28 15:34:31.832556', count=30, dataset='kandinsky', debug=False, device=device(type='cuda'), embedding_dim=1024, entity='', entropy=False, exp_decay=0.9, extractor_training_epochs=20, gamma=0.001, imp_op='Prod', iterations=100, joint=False, lr=0.001, model='kandslsinglejoint', n_epochs=40, n_support=75, non_verbose=False, notes=None, num_distinct_labels=3, num_q

# TRAINING LOOP

In [ ]:
def train(model: MnistDPL, dataset: BaseDataset, _loss: ADDMNIST_DPL, args, save_folder: str, patience: int = 5):
    
    best_f1 = 0.0
    epochs_no_improve = 0   # for early stopping

    model.to(model.device)

    train_loader, val_loader, test_loader = dataset.get_data_loaders()
    dataset.print_stats()
    scheduler = torch.optim.lr_scheduler.ExponentialLR(model.opt, args.exp_decay)
    w_scheduler = None
    if args.warmup_steps > 0:   w_scheduler = GradualWarmupScheduler(model.opt, 1.0, args.warmup_steps)

    fprint("\n--- Start of Training ---\n")

    # default for warm-up
    model.opt.zero_grad()
    model.opt.step()

    # ^ Start of training
    for epoch in range(args.n_epochs):
        model.train()

        ys, y_true, cs, cs_true = None, None, None, None

        for i, data in enumerate(train_loader):
            images, labels, concepts = data
            images, labels, concepts = (
                images.to(model.device),
                labels.to(model.device),
                concepts.to(model.device),
            )

            out_dict = model(images)
            out_dict.update({"LABELS": labels, "CONCEPTS": concepts})
            
            model.opt.zero_grad()
            loss, losses = _loss(out_dict, args)

            loss.backward()
            model.opt.step()

            if ys is None:
                ys = out_dict["YS"]
                y_true = out_dict["LABELS"]
                cs = out_dict["pCS"]
                cs_true = out_dict["CONCEPTS"]
            else:
                ys = torch.concatenate((ys, out_dict["YS"]), dim=0)
                y_true = torch.concatenate((y_true, out_dict["LABELS"]), dim=0)
                cs = torch.concatenate((cs, out_dict["pCS"]), dim=0)
                cs_true = torch.concatenate((cs_true, out_dict["CONCEPTS"]), dim=0)

            if i % 10 == 0:
                progress_bar(i, len(train_loader) - 9, epoch, loss.item())

        y_pred = torch.argmax(ys, dim=-1)
        #print("Argmax predictions have shape: ", y_pred.shape)

        if "patterns" in args.task:
            y_true = y_true[:, -1]  # it is the last one

        model.eval()
        tloss, cacc, yacc, f1 = evaluate_metrics(model, val_loader, args)

        # update the (warmup) scheduler at end of the epoch
        if epoch < args.warmup_steps:
            w_scheduler.step()
        else:
            scheduler.step()
            if hasattr(_loss, "grade"):
                _loss.update_grade(epoch)

        ### LOGGING ###
        fprint("  ACC C", cacc, "  ACC Y", yacc, "F1 Y", f1)
        print()

        if not args.tuning and f1 > best_f1:
            print("Saving...")
            # Update best F1 score
            best_f1 = f1
            epochs_no_improve = 0

            # Save the best model
            torch.save(model.state_dict(), save_folder)
            print(f"Saved best model with F1 score: {best_f1}")
            print()
        
        elif f1 <= best_f1:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    
    fprint("\n--- End of Training ---\n")
    return best_f1

# RUN ALL THINGS

In [10]:
f1_scores = dict()
for seed in args.seeds:
    print(f"*** Training model with seed {seed}")
    print("Chosen device:", model.device)
    if not os.path.exists(save_path): os.makedirs(save_path, exist_ok=True)
    save_folder = os.path.join(save_path, f"{save_model_name}_{seed}.pth")
    print("Saving model in folder: ", save_folder)

    best_f1 = train(model=model,
          dataset=dataset,
        _loss=loss,
        args=args,
        save_folder=save_folder,
    )
    f1_scores[(seed)] = best_f1
    save_model(model, args, seed)  # save the model parameters
    
    print(f"*** Finished training model with seed {seed}")

print("Training finished.")
best_weight_seed = max(f1_scores, key=f1_scores.get)
print(f"Best weight and seed combination: {best_weight_seed} with F1 score: {f1_scores[best_weight_seed]}")

*** Training model with seed 0
Chosen device: cuda
Saving model in folder:  ../outputs/kand/baseline-DEBUG/sl/sl_0.pth
Loaded datasets in 4.444101095199585 s.
Len loaders: 
 train: 4000 
 val: 1000
 len test: 1000
## Statistics ##
Train samples 4000
Validation samples 1000
Test samples 1000

--- Start of Training ---

Labels:  torch.Size([32, 4])
Concepts:  torch.Size([32, 3, 6])


RuntimeError: Debug stop